### install requirements

In [ ]:
# !pip3 install torch torchvision torchaudio

# !pip install ultralytics






#### load & test model

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")


In [ ]:
# test model on image
img = "media/sample.jpg"
results = model(source=img, save=True)


In [ ]:
#test real time detection on video
video = 'media/sample.mp4'
results = model.track(source=video, show=True)

Augmentation

In [ ]:
# !pip install albumentations==1.* 

import os, cv2, glob, shutil
import albumentations as A

# paths
IMG_DIR = "data/images/train"
LBL_DIR = "data/labels/train"
OUT_IMG_DIR = "data/images/train_augmented"
OUT_LBL_DIR = "data/labels/train_augmented"

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LBL_DIR, exist_ok=True)

transform = A.Compose([
    A.RandomBrightnessContrast(p=0.7),
    A.RandomGamma(p=0.5),
    A.RandomShadow(p=0.4),
    A.MotionBlur(p=0.3),
])

image_paths = sorted([p for p in glob.glob(os.path.join(IMG_DIR, "*"))
                      if os.path.splitext(p)[1].lower() in [".jpg", ".jpeg", ".png"]])

aug_count = 0
for img_path in image_paths:
    fname = os.path.basename(img_path)
    lbl_path = os.path.join(LBL_DIR, os.path.splitext(fname)[0] + ".txt")
    if not os.path.exists(lbl_path):
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue

    aug_img = transform(image=img)["image"]

    out_img_name = f"aug_{fname}"
    out_lbl_name = f"aug_{os.path.splitext(fname)[0]}.txt"

    cv2.imwrite(os.path.join(OUT_IMG_DIR, out_img_name), aug_img)

    shutil.copy2(lbl_path, os.path.join(OUT_LBL_DIR, out_lbl_name))

    aug_count += 1

print(f"Augmented samples saved: {aug_count}")


#### train custome model for drawsiness detection

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")  

#train model with dataset
model.train(data="dataset.yaml", epochs=50, imgsz=640, batch=16)


In [ ]:
# Evaluation on test set
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/best.pt")  

metrics = model.val(
    data="dataset.yaml",
    split="test"
)

print(metrics)  


In [ ]:
#test trained model on video

from ultralytics import YOLO

#path to your best.pt (trained model output)
model = YOLO("runs/detect/train/weights/best.pt")


model.predict(
    source="sample.mp4",       #path to sample video
    save=True,                      
    conf=0.3,                       
    show=False,       
    project="video_output",      #save folder path                      
    name=".",      
    exist_ok=True                  
)

In [ ]:
#filter drowsiness detection with consecutive-frame confirmation
#(only for video) 
import cv2
from ultralytics import YOLO

model = YOLO("drowsiness_detection.pt")
cap = cv2.VideoCapture("sample.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter("drowsy_filtered_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

drowsy_counter = 0
drowsy_threshold = 15

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.4, verbose=False)
    boxes = results[0].boxes
    names = model.names

    classes = boxes.cls.cpu().numpy()
    drowsy_detected = any(names[int(cls)] == "drowsy" for cls in classes)

    if drowsy_detected:
        drowsy_counter += 1
    else:
        drowsy_counter = 0

    if drowsy_counter >= drowsy_threshold:
        annotated = results[0].plot()
        cv2.putText(annotated, "DROWSINESS ALERT!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        out.write(annotated)
    else:
        out.write(frame)  

cap.release()
out.release()
